In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cross_decomposition import PLSRegression
import os.path
import glob
from scipy.io import loadmat
import pandas as pd
from scipy import signal
from scipy.signal import butter, sosfilt
from scipy.interpolate import interp1d
from scipy import signal
from scipy.signal import butter, sosfilt

In [20]:
def get_ecog_and_motions(dir):
    n_ch = len(glob.glob(os.path.join(dir, 'ECoG_ch*.mat')))
    ECoG = []
    for ch in range(1, n_ch + 1):
        ECoGData = loadmat(os.path.join(dir, 'ECoG_ch%d.mat' % ch))
        ECoGData = ECoGData['ECoGData_ch%d' % ch]
        ECoG.append(ECoGData[0])

    ECoG = np.array(ECoG)

    Motion = loadmat(os.path.join(dir, 'Motion.mat'))
    MotionTime = Motion['MotionTime']
    Motion = Motion['MotionData']
        
    LSHO = Motion[0,:]
    LELB = Motion[1,:]
    LWRI = Motion[2,:]
    RSHO = Motion[3,:]
    RELB = Motion[4,:]
    RWRI = Motion[5,:]
    
    Time = loadmat(os.path.join(dir, 'ECoG_time.mat'))
    Time = Time['ECoGTime']

    return ECoG.T, LSHO[0], LELB[0], LWRI[0], RSHO[0], RELB[0], RWRI[0], Time.T, MotionTime.T

In [21]:
def extract_needed_data(ECoG, LSHO, LELB, LWRI, RSHO, RELB, RWRI, Time, MotionTime):
    
    body_center = (LSHO - RSHO)/2 #It is a static point in this experiment
    motion_left_hand = LWRI - body_center #centering motion
    
    return ECoG, motion_left_hand

In [22]:
dir = 'Data\\20100802S1_ECoG_Motion6'

In [23]:
ECoG, LSHO, LELB, LWRI, RSHO, RELB, RWRI, time, motion_time =  get_ecog_and_motions(dir)

In [6]:
x, y = extract_needed_data(ECoG, LSHO, LELB, LWRI, RSHO, RELB, RWRI, time, motion_time)

In [7]:
signal_data = np.c_[time, x]
motion_data = np.c_[motion_time, y]

In [28]:
def synchronize_interpol(signal_data, motion_data): 
    start = max(signal_data[1, 0],motion_data[1,0])
    end = min(signal_data[-1, 0],motion_data[-1,0])

    #cutting signal and motion, only overlapping time left
    signal_data = signal_data[:,:][(signal_data[:,0]>=start)]
    signal_data = signal_data[:,:][(signal_data[:,0]<=end)]
    motion_data = motion_data[:,:][motion_data[:,0]>= start] 
    motion_data = motion_data[:,:][motion_data[:,0]<= end]
    M = []
    #signal and motion have different time stamps, we need to synchronise them
    #interpolating motion and calculating arm position in moments of "signal time"
    for i in range(1,motion_data.shape[1]):
        interpol = interp1d(motion_data[:,0],motion_data[:,i],kind="cubic")
        x = interpol(signal_data[:,0])
        M.append(x)

    #downsampling in 10 times to get faster calcultions

    ecog_signal = signal_data[::10,1:]
    motion = np.array(M).T[::10,:]
    time = signal_data[::10,0]
    
    
    #self.signal = signal_data[:,1:]
    #self.motion = np.array(M).T[:,:]
    #self.time = signal_data[:,0]

    return ecog_signal, motion, time

In [29]:
#signal filtering (not sure that it works correctly)
def bandpass_filter(ecog_signal, lowcut, highcut, fs = 100, order=7):
    nyq =  fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = signal.butter(order,  (low, high), btype='band',analog=False,output='sos')
    filtered_signal = np.array([sosfilt(sos, ecog_signal[:,i]) for i in range(ecog_signal.shape[1])])

    return filtered_signal.T

    #Generating a scalogram by wavelet transformation 
def scalo(ecog_signal, motion, time, window, freqs,start,end, step = 100): #window in sec,freqs in Hz, step in ms
    #div = 1
    X = ecog_signal[start:end,:]
    div = 10 #downsampling
    window_len = int(((window * 1000 // step) + 2) * step//div)
    scalo = np.empty((X.shape[0]-window_len,X.shape[1],freqs.shape[0],(window * 1000 // step) + 2))
    for i in range(X.shape[1]):
        for j in range(window_len,X.shape[0]):
            scalo[j-window_len,i,:,:] = signal.cwt(data = X[j-window_len:j,i],
                                                    wavelet=signal.morlet,widths = freqs)[:,::step//div] **2
    return scalo, motion[start+window_len:end,:], time[start+window_len:end]
    

In [30]:
ecog_signal, motion, time = synchronize_interpol(signal_data, motion_data)
filtered_signal = bandpass_filter(ecog_signal, lowcut=0.2,highcut=60,fs = 100) 

In [ ]:
len(filtered_signal)

In [31]:
%%time
#works for a long time, needs about 1 hour to preprocess 15 min experiment
freq = np.array([i for i in range(10,150,15)]) 
scalo,motion,time = scalo(ecog_signal, motion, time, 1,freq,start = 0,end = 104000) # preprocess first 3000(30 sec) objects of data

C:\Users\User\Anaconda3\lib\site-packages\scipy\signal\wavelets.py:364: ComplexWarning: Casting complex values to real discards the imaginary part
  mode='same')


Wall time: 1h 54min 7s


In [32]:
X = scalo.reshape((scalo.shape[0],scalo.shape[1]*scalo.shape[2]*scalo.shape[3]))

In [ ]:
model = PLSRegression(n_components = 80)
model.fit(X[0:10000],motion[0:10000])

In [ ]:
Y_pred = model.predict(X[0:10000])

In [ ]:
plt.figure(figsize = (15,6))
plt.plot(time[0:10000],Y_pred[:,0], color = 'orange')
plt.plot(time[0:10000],motion[0:10000,0], color = 'blue')

In [ ]:
from sklearn.metrics import r2_score
r2_score(motion[:10000], Y_pred)